<a href="https://colab.research.google.com/github/PoiZeX/CloudCourse/blob/main/PyProject/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from ipywidgets.widgets.widget_button import ButtonStyle
#@title
import time
from ipywidgets.widgets.widget_style import Style
from zmq.constants import NULL
from firebase import firebase
from IPython.display import display, Audio
import IPython
from google.colab import output
import random
import ipywidgets as widgets
from IPython import display
from IPython.display import display, HTML, clear_output, Image
from ipywidgets import VBox, HBox, HTML, Layout, Box, Label, widgets
import matplotlib.pyplot as plt
import math
import re
import requests
import io
import base64
from datetime import date


# Add the custom CSS styling for the button
style = """
<style>
    .custom-button {{
        border: none;c
        color: white;
        text-align: center;
        display: inline-block;
        font-size: 16px;
        margin: 4px 2px;
        cursor: pointer;
        border-radius: 12px;
        padding: 10px 24px;
        background-color: #4CAF50;
    }}
</style>
"""

display(HTML(style))


##########################################################################################################################
############################################### CONFIGURATION <-  ########################################################
##########################################################################################################################

FIREBASE_PATH = 'https://ex6game-default-rtdb.europe-west1.firebasedatabase.app/'
style = {'description_width': '150px'}
box_layout = widgets.Layout(display='flex',  flex_flow='column',  align_items='center',  width='100%')
################################################################################
#################################### CLASSES ###################################
################################################################################
class Question():
  answers = []
  def __init__(self, question, lst_options, correct):
    self.question = question
    self.answers = lst_options
    self.correct = correct

  def isCorrect(self, str_option):
    correctAns = self.answers[int(self.correct)-1]
    return str_option == str(correctAns)

  def get_random_answers(self):
      return random.sample(self.answers, len(self.answers))

  def get_question_str(self):
    return self.question

  def get_all_answers(self):
    return self.answers

  def get_correct_inx(self):
    return self.correct




  def set_question_str(self, question_str):
    self.question = question_str

  def set_answers(self, answers_lst):
    self.answers = answers_lst

  def set_correct(self, correct_inx):
    self.correct = correct_inx

################################################################################
class QuestionsDB():
  lst_questions = []
  current_question = None
  original_questions_lst = []

  def __init__(self, lst_questions):
    self.lst_questions = [Question(item['Question'], [item['Option1'], item['Option2'],item['Option3'],item['Option4']], item['Correct']) for item in lst_questions if item is not None]
    self.original_questions_lst = self.lst_questions.copy()

  def get_random_question(self):
    global randomQuestionAndAnswer, answers, question, radioButtonLst
    randomQuestionAndAnswer = NULL # we want to generate new item...
    if self.lst_questions:
      random_question = random.sample(self.lst_questions, 1)[0]
      self.lst_questions.remove(random_question)
      self.current_question = random_question
      return random_question
    else:
      self.current_question = []
      return

  def add_question_to_DB(self):    pass

  def add_question_to_local_list(self,question):
    newQuestion = Question(question['Question'],[question['Option1'], question['Option2'],question['Option3'],question['Option4']],question['Correct'])

    self.lst_questions.append(newQuestion)
    self.original_questions_lst = self.lst_questions


  def remove_question_from_DB(self, question):
   # Remove the module from the Firebase database
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH, None)
    res = FBconn.get('/Questions', None)

    count = -1
    if res:
        for key in res:
            count = count + 1
            if (key):
                if key['Question'] == question.get_question_str():
                    remove_url = f'/Questions/{count}'
                    FBconn.delete(remove_url, None)
                    # print("Collection removed successfully from the database.")
                    break
        else:
            print("Failed to find the collection in the database.")

    # Remove the module from the local list
    for module in self.lst_questions:
        if module.get_question_str() == question.get_question_str():
            self.lst_questions.remove(module)
            self.original_questions_lst = self.lst_questions
            break
    else:
        print("Failed to find the module in the local list.")

  def get_num_of_questions(self):
    return len(self.lst_questions)

  def get_original_len(self):
    return len(self.original_questions_lst)

  def get_all_questions(self):
    return self.original_questions_lst



  def get_current_question_obj(self):
    return self.current_question
################################################################################

class StatsDB():
  lst_stats = []

  def __init__(self, lst_stats):
    self.lst_stats = [Score(item['Date'], item['Username'], item['Stats'],  item['Points']) for item in lst_stats if item is not None]

  def getAllStats(self):
    return self.lst_stats

  def getStatsByUserid(self, userId):
    pass

  def getStatsByDateRange(self, startDate, endDate):
    pass


################################################################################
class User():
  def __init__(self, username, password, role, totalPoints):
    self.username = username
    self.password = password
    self.role = role
    self.totalPoints = totalPoints

  def isAdmin(self):
    return self.role == "Admin"

  def getName(self):
    return self.username

  def getTotalPoints(self):
    return self.totalPoints


################################################################################

class Score():
  def __init__(self, date, username, statsLst, points):
    self.date = date
    self.username = username
    self.statsLst = statsLst
    self.points = points

  def getStatsLst(self):
    return self.statsLst

  def getGamePoints(self):
    return self.points

##########################################################################################################################
#############################################  FIREBASE #################################################################
##########################################################################################################################
def get_questions_from_db():
  global questionsFromDB
  FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
  res = FBconn.get('/Questions/',None)

  if res:
    # insert questions to DB class
    questionsFromDB = QuestionsDB([question for question in res])

def get_stats_from_db():
  global statsFromDB
  FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
  res = FBconn.get('/Scores/',None)

  if res:
    # insert questions to DB class
    statsFromDB = StatsDB([stat for stat in res if res is not None])


def incrementPointsToUser(username, points_to_increment):
  FBconn = firebase.FirebaseApplication(FIREBASE_PATH, None)

  # Query the database to find the user with the specified username
  users = FBconn.get('/Users', None)
  user_id = None

  for user_key in users: # find the key
      if users[user_key]['Username'] == username:
          user_id = user_key
          break

  if user_id:
      # Get the current total points of the user
      current_points = users[user_id]['TotalPoints']

      # Increment the total points by the desired amount
      new_points = current_points + points_to_increment

      # Update the total points in the database using FBconn.put
      FBconn.put('/Users/' + user_id, 'TotalPoints', new_points)

      print(f"Total points for user '{username}' updated to: {new_points}")
  else:
      print(f"No user found with the username '{username}'")

def insert_new_stat_to_DB(username, statsLst, gamePoints):
    current_date = date.today()
    formatted_date = current_date.strftime("%Y-%m-%d")
    stats = {
        'Date': formatted_date,
        'Username': username,
        'Stats': statsLst,
        'Points': gamePoints
    }


    # Insert the new item into the "Stats" database using FBconn.put
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
    result = FBconn.post('/Scores', stats)

def get_user_stats(username):
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH, None)
    scores = FBconn.get('/Scores', None)

    user_stats = []
    if scores:
        # Iterate through each score entry
        for score_key, score_value in scores.items():
            # Check if the username matches
            if score_value.get('Username') == username:
                stats = score_value.get('Stats')
                if stats:
                    user_stats.extend(stats)

    return user_stats
##########################################################################################################################
######## GLOBAL VARIABLES
##########################################################################################################################
questionsFromDB = None
StatsFromDB = None
btn_go_back = widgets.Button(description="Back", button_style="primary",layout=Layout(width="80px"))
current_user = NULL
points = 0
quantity_of_questions=0
quantity_of_correct_questions=0
list_of_stats = []  # will be filled with number of attempts for EACH question
question = ""
buttonsLst = []
NumberOfQuestionsToPlay = 4
#For Admin Form
#--->
cb_correctAnswerId = NULL
tb_question= NULL
tb_answer1 = NULL
tb_answer2 = NULL
tb_answer3 = NULL
tb_answer4 = NULL
offset = 0
#<---

##########################################################################################################################
########################################################## UTILS #########################################################
##########################################################################################################################
from enum import Enum
class WidgetEnum(Enum):
    BUTTON = 1
    LABEL = 2
    TEXTBOX = 4
    PASSWORD = 8

def show_h2_center_text(text):
  """
  Generic function for h2 centered text
  """
  display(IPython.display.HTML(f"<div style='color: white; text-align: center; justify-content: center; align-items: center;'>"+
    f"<h2 id='h2Msg' style='color:white;'>{text}</h2><br/>"))

def show_h4_center_text(text):
  """
  Generic function for h4 centered text
  """
  display(IPython.display.HTML(f"<div style='color: white; text-align: center; justify-content: center; align-items: center;'>"+
    f"<h4 id='h4Msg' style='color:white;'>{text}</h4><br/>"))


def create_widget(widget_enum, description, button_style = "", layout="", onclick=""):
  b_layout = widgets.Layout( width='auto', height='auto', min_width="150px", margin="20px 0px 0px 110px")
  b_style={'white_space': 'pre-wrap'}
  style = {'description_width': '150px'}

  obj = None
  if widget_enum == WidgetEnum.BUTTON:
    if layout != "":
      b_layout = layout
    obj = widgets.Button(description = str(description), button_style = button_style,layout = b_layout, style=b_style)
    if onclick != "":
        obj.on_click(onclick)

  elif widget_enum == WidgetEnum.LABEL:
    return None

  elif widget_enum == WidgetEnum.PASSWORD:
      obj = widgets.Password(description=description, style=style)

  elif widget_enum == WidgetEnum.TEXTBOX:
      obj = widgets.Text(description=description, style=style)

  else:
    return None

  return obj

def center_widgets(widgets_lst):
  """
  gets a list of widgets and centered them
  """
  return widgets.HBox(children=widgets_lst,layout=box_layout)

def show_go_back(back_event):
  if btn_go_back != NULL:
    display(btn_go_back)
    btn_go_back.on_click(back_event)


##########################################################################################################################
################################### RESULTS SECTION -> SHOWS THE RESULTS AND GRAPH <-  ###################################
##########################################################################################################################

def show_user_statistics(combined_list):
    """
    Generate and show statistics plot: Number of games
    """
    count_dict = {i: combined_list.count(i) for i in range(5)}
    labels = ['#0', '#1', '#2', '#3', '#4']
    sizes = [count_dict[i] for i in range(5)]
    explode = (0, 0, 0, 0, 0)

    fig, ax = plt.subplots()
    wedges, _, _ = ax.pie(sizes, explode=explode, labels=None, autopct='', shadow=True, startangle=90)
    ax.axis('equal')
    plt.title('Distribution of total attempts')

    # Create legend
    legend_labels = [f'{label}: {size}' for label, size in zip(labels, sizes)]
    plt.legend(wedges, legend_labels, loc='center', bbox_to_anchor=(0.5, -0.1), ncol=5, title='Legend')

    # Add margin to the bottom of the chart
    plt.subplots_adjust(bottom=0.2)

    # Save the chart to a buffer
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)

    # Encode the chart image as base64 string
    chart_image = base64.b64encode(buffer.getvalue()).decode()

    # Generate the HTML string for the chart
    chart_html = f'<img src="data:image/png;base64,{chart_image}" style="max-width: 85%; height: auto;">'

    plt.close(fig)  # Close the figure to free up memory

    return chart_html


def show_statistics_chart():
    """
    Generate and show statistics plot.
    The statistics are the number of attempts for each question.
    """
    global list_of_stats
    list_of_stats.insert(0, 0)  # normalize the data
    fig, ax = plt.subplots()  # configure

    ax.plot(list_of_stats)  # make the line plot

    # mark the dots
    for i, stat in enumerate(list_of_stats):
        if stat != 0:
            ax.scatter(i, stat, color='green', marker='o')

    # make data readable (start from 1)
    ax.set_xticks(range(1, len(list_of_stats)))
    ax.set_yticks(range(1, 5))

    ax.set_title("Your game statistics")
    ax.set_ylabel("Number of attempts")
    ax.set_xlabel("Question #")

    # Save the chart to a buffer
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)

    # Encode the chart image as base64 string
    chart_image = base64.b64encode(buffer.getvalue()).decode()

    # Generate the HTML string for the chart
    chart_html = f'<img src="data:image/png;base64,{chart_image}" style="max-width: 85%; height: auto;">'
    plt.close(fig)  # Close the figure to free up memory
    return chart_html


def show_results_page():
    audio_file = 'https://www.soundjay.com/human/sounds/applause-4.mp3'
    display(HTML(f"<audio src='{audio_file}' autoplay></audio>"))

    good_words = ""
    badge_src = ""
    btn_return_menu = create_widget(WidgetEnum.BUTTON, "Return to Menu", onclick=show_menu_page)
    btn_return_menu.style.button_color = 'green'
    btn_return_menu.layout.margin = "20px auto"
    btn_return_menu.layout.justify_content = 'center'
    btn_return_menu.layout.align_items = 'center'
    total_score_to_earn = 3 * questionsFromDB.get_num_of_questions()
    name = current_user.getName()

    if total_score_to_earn < 2:
        good_words = f"{name}, You are AWESOME!"
        badge_src = "https://cdn.pixabay.com/photo/2016/08/26/15/56/medal-1622523_960_720.png"
    elif total_score_to_earn - points < 15:
        good_words = f"{name}, You are great! Try again to beat yourself"
        badge_src = "https://cdn.pixabay.com/photo/2016/08/26/15/59/medal-1622529_960_720.png"
    else:
        good_words = f"{name}, You are good, but need more practice. Try out our courses and play again"
        badge_src = "https://cdn.pixabay.com/photo/2016/08/26/16/04/medal-1622549_960_720.png"

    succ_percentages = round((points / total_score_to_earn) * 100)

    user_statistics_area =  ""

    if current_user.getName() != "Guest":
      # gets all user statistics

      # combine to 1 list

      user_statistics_area = f"""
        <div style="width: 500px; height: 300px; margin: 0 auto;">
        {show_user_statistics(get_user_stats(current_user.getName()))}
        </div>
      """




    display(HTML(f"""
        <div style="background-image: url('https://cdn.pixabay.com/photo/2017/08/09/13/49/cartoon-2614617_960_720.jpg');
                    background-size: cover; background-repeat: no-repeat; overflow: auto; height: 90vh; display: flex;
                    align-items: center; justify-content: center;">
            <div style="background-color: rgba(0, 0, 0, 0.8); border-radius: 10px; padding: 40px; max-width: 800px;
                        text-align: center; color: white; font-family: Arial, sans-serif;">
                <h2 style="color: red; font-size: 36px;">
                    <img src="https://www.animatedimages.org/data/media/492/animated-fireworks-image-0087.gif"
                        style="width: 40px; height: 40px;"/> Game over!
                    <img src="https://www.animatedimages.org/data/media/492/animated-fireworks-image-0087.gif"
                        style="width: 40px; height: 40px;"/>
                </h2>
                <h4 style="text-decoration: underline; font-size: 16px; color: white;">Your score</h4>
                <br/>
                <div style="font-size: 16px;">
                    <strong>Points:</strong> {points}
                </div>
                <div style="font-size: 16px;">
                    <strong>Correct answers:</strong> {quantity_of_correct_questions}
                </div>
                <br/>
                <strong style="font-size: 16px;">You could earn up to {total_score_to_earn} points.</strong>
                <br/>
                <strong style="font-size: 16px;">Your success rate is: {succ_percentages}%</strong>
                <h4 style="color: #00bfff; font-weight: bold; font-size: 16px;">{good_words}</h4>
                <br/>
                <div style="display: flex; flex-direction: column; align-items: center;">
                    <div style="margin-bottom: 20px;">
                        <span style="display: block;">You earned a new badge:</span>
                        <img src="{badge_src}" style="width: 100px; height: 100px; margin: 0 auto;">
                    </div>
                    <div style="width: 500px; height: 300px; margin: 0 auto;">
                        {show_statistics_chart()}
                    </div>
                    {user_statistics_area}
                </div>
            </div>
        </div>
    """))


    display(btn_return_menu)



##################################################################################################################################
################################### GAME SECTION -> QUESTION PROGRESS FROM START TO FINISH <-  ###################################
##################################################################################################################################

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=NumberOfQuestionsToPlay,
    bar_style='info',
    style={'bar_color': '#19b7cf'},
    orientation='horizontal',
)
description_html = HTML('<font color="white" face="Courier New, monospace" style="font-size: 15px;"><b>Progress:</b></font>')
progress_bar.layout.margin = '0 5px 0 0'
description_html.layout.margin = '0 0 0 5px'

def reset_game():
    global points, quantity_of_questions, quantity_of_correct_questions, list_of_stats, questionsFromDB
    points = 0
    quantity_of_questions = 0
    quantity_of_correct_questions = 0
    list_of_stats = []
    questionsFromDB = None
    progress_bar.value=0

def game_over(btn=NULL):
    output.clear()
    if current_user.getName() != "Guest":
      incrementPointsToUser(current_user.getName(), points)
      insert_new_stat_to_DB(current_user.getName(), list_of_stats, points)
    show_results_page()
    reset_game()

def on_continue_button_clicked(b):
    global success_box
    global continue_button
    continue_button.button_style = 'warning'
    continue_button.disabled = True

#    success_box.close() # Close the box after clicking continue
    update_total_questions_quantity()
    if questionsFromDB.get_original_len() - questionsFromDB.get_num_of_questions() == NumberOfQuestionsToPlay:
        game_over()
    else:
        show_questions_page()

button_layout = widgets.Layout(width='150px', height='40px')
# continue_button_layout = widgets.Layout(width='250px', height='40px')

continue_button = widgets.Button(description='Continue', disabled=True, button_style='warning', layout=button_layout)
continue_button.on_click(on_continue_button_clicked)
#continue_button = create_widget(WidgetEnum.BUTTON, 'Continue', '', '', on_continue_button_clicked, disabled=True)
#continue_button.style.button_color = 'red'

btn_stop_game = create_widget(WidgetEnum.BUTTON, "Stop Game", onclick=game_over, layout=button_layout)
btn_stop_game.style.button_color = 'red'
btn_stop_game.layout.margin = "0 0 0 auto"  # Adjust the top margin value here

container = widgets.HBox([btn_go_back,continue_button,btn_stop_game], layout=widgets.Layout(justify_content='center', width='100%'))
container.layout.margin = "20px 0 0 auto"


def display_questions():
    """
    Create questions page elements and show them
    """
    global randomQuestionAndAnswer, answers, question, radioButtonLst

    current_question = questionsFromDB.get_current_question_obj()

    # Define the CSS style
    style = """
    <style>
        body {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            margin: 0;
            background-color: #2B2B2B;
            color: #ECECEC;
            font-family: 'Courier New', monospace;
            min-height: 80vh;
            padding: 20px;
        }

        .question-container {
            text-align: center;
            margin-bottom: 20px;

        }

        .question-title {
            font-size: 24px;
            font-weight: bold;
            margin-bottom: 10px;
        }

        .points-label {
            font-size: 16px;
            font-weight: bold;
            margin-bottom: 10px;
            color: white;
        }

      .answer-button {
          border: none;
          text-align: center;
          font-size: 15px;
          margin: 4px 2px;
          cursor: pointer;
          border-radius: 8px;
          padding: 8px 16px;
          font-family: 'Courier New', monospace;
          background-color: #cf7a19;
          font-weight: bold;
          white-space: normal;
          word-wrap: break-word;
          height: 80px;   /* Increase the height of the button */
          overflow: hidden;  /* Ensure that extra content will not overflow */
          display: flex;  /* Use flex to align content */
          align-items: center;  /* Vertically align the content in the center */
          justify-content: center;  /* Horizontally align the content in the center */
      }

        .card {
            background-color: #3E3D32;
            padding: 10px;
            width: 50%;
            margin: 0 auto;
            border-radius: 8px;
            box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1);
            text-align: center;
        }
    </style>
    """
    # Display the question
    display(IPython.display.HTML(f"<div class='card'><h2 style='display: flex; justify-content: center; align-items: center; color: white; font-family: Courier New, monospace;'>{current_question.get_question_str()}</h2></div>"))

    # Display points label with icon
    points_label = HTML(f"<div class='points-label'><img src='https://cdn3.iconfinder.com/data/icons/cyber-monday-astute-vol-1-1/512/Earn_Reward_Points-512.png' height='34px' width='34px' style='vertical-align: middle;'> <strong>Points:</strong> {points}</div>")
    display(points_label)


    # Create an HBox container and center align the widgets
    container_proggres = widgets.HBox([description_html, progress_bar], layout=widgets.Layout(justify_content='center'))
    display(container_proggres)

    buttonsLst = [create_widget(WidgetEnum.BUTTON, str(answer), onclick=on_answer_button_clicked)\
                  for answer in current_question.get_random_answers()]
#sharon
    # Apply the custom button style
    for btn in buttonsLst:
        btn.layout.height = '80px'
        btn.add_class("answer-button")

    left_box = VBox([buttonsLst[0], buttonsLst[1]], layout=Layout(width='600px'))
    right_box = VBox([buttonsLst[2], buttonsLst[3]], layout=Layout(width='600px'))

   # Display the question buttons with the custom CSS style
    display(widgets.HTML(style))
    display(widgets.HBox([left_box, right_box]))
    display(container)

error_output = widgets.Output()
def show_questions_page():
  """
  Generate a new question
  """
  global show_go_back
  output.clear()
  questionsFromDB.get_random_question()
  display_questions()
  display(error_output)
  btn_go_back.description = "Back"

# Initialize the success box
success_box = None

def on_answer_button_clicked(b):
    """
    The true logic for answer button clicked
    """
    global list_of_stats, NumberOfQuestionsToPlay, success_box, error_output

    list_of_stats[questionsFromDB.get_original_len() - questionsFromDB.get_num_of_questions()-1] += 1

    if b.description == "Stop Game":
        game_over()
        return

    # check if user's answer is correct
    if questionsFromDB.current_question.isCorrect(str(b.description)):
        if is_first_try():
            display(HTML('<div style="text-align:center;"><br/><h2 style="color:green; font-family: Courier New, monospace">On the first try! Great job!</h2><br/></div>'))
        else:
            display(HTML('<div style="text-align:center;"><br/><h4 style="font-family: Courier New, monospace; color: white">Correct!</h4><br/></div>'))
        update_points(3, display_points=True)
        update_total_correct_questions_quantity()
        b.style.button_color = 'lightgreen'

        # show success box instead of sleep
        show_success_dialog()
    else:
        update_points(-1, display_points=False)
        b.style.button_color = 'red'

        # clear previous error message
        error_output.clear_output()

        # display new error message
        with error_output:
            display(HTML('<div style="text-align:center;"><br/><h4 style="font-family: Courier New, monospace; color: white;">Incorrect answer<br/>Try Again! You can do it!</h4><br/></div>'))
def show_success_dialog():
    global continue_button
    """
    Show a success dialog with a continue button. After continue is clicked,
    proceed to the next question or end the game if all questions have been answered.
    """
    global success_box
    global error_output
    error_output.clear_output()
    continue_button.button_style = 'success'
    continue_button.disabled = False


def is_first_try():
    """
    Check if it's the user's first attempt on the current question
    """
    return list_of_stats[questionsFromDB.get_original_len() - questionsFromDB.get_num_of_questions() - 1] == 1

def update_points(new_points, display_points=True):
    """
    Update and display current points
    """
    global points
    points += new_points
    if display_points:
        points_html = f'<div style="text-align:center;"><b>New Points:</b> {points}</div>'
        display(IPython.display.HTML(points_html))

def update_total_questions_quantity():
    global quantity_of_questions
    quantity_of_questions+=1

def update_total_correct_questions_quantity():
    global quantity_of_correct_questions
    quantity_of_correct_questions+=1
    progress_bar.value=quantity_of_correct_questions




##########################################################################################################################
##########################################################ADMIN ##########################################################
##########################################################################################################################

#################### background AdminEditPage# Define the HTML and CSS for the background style ####################
background_html = """
<style>
body {
    background-color: #3E3D32;
}
</style>
"""
#################### background AdminEditPage# Define the HTML and CSS for the background style ####################
#sharonnew
def insert_new_question_to_db(b):
    """
    Inserting new question. Return true if succeeded
    """
    global tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4, cb_correctAnswerId, offset, error_widgets, success_message

    fields = [tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4]
    error_label.value = ''
    success_message.value = ''  # Reset success message when function is called

    for field, error_widget in zip(fields, error_widgets):
        field.layout.border = ''  # Reset border styles when page is shown
        error_widget.value = ''  # Reset error widgets

    if any(field.value.strip() == '' for field in fields):
        for field, error_widget in zip(fields, error_widgets):
            if field.value.strip() == '':
                error_widget.value = "<span style='color: red;'>X</span>"  # Show 'X' if field is empty
        error_label.value = "<span style='color: red;'>All fields must be filled in.</span>"
        return False

    try:
        data_to_upload = {
            'Question': tb_question.value,
            'Option1': tb_answer1.value,
            'Option2': tb_answer2.value,
            'Option3': tb_answer3.value,
            'Option4': tb_answer4.value,
            'Correct': cb_correctAnswerId.value
        }

        FBconn = firebase.FirebaseApplication(FIREBASE_PATH, None)
        offset += 1

        result = FBconn.put("/Questions", name=f"{len(questionsFromDB.get_all_questions())+offset+1}", data=data_to_upload)
        success_message.value = "<div style='text-align: center; background-color: #d4edda; color: #155724; width: 400px; border: 1px solid #c3e6cb; padding: 10px;'>Question successfully added!</div>"
        time.sleep(4)
        success_message.value = ''  # Clear success message after 2 seconds
        for field in fields:
            field.value = ''
        cb_correctAnswerId.value = '1'  # Reset the dropdown to the first option
        questionsFromDB.add_question_to_local_list(data_to_upload)
        return True
    except Exception as e:
        print(f"Can't insert into DB\n{e}")
        return False

def show_add_questions_page(b):
    """
    Present a form for adding new questions into DB.
    """
    global cb_correctAnswerId, tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4, offset, error_label, error_widgets, success_message
    output.clear()

    cb_correctAnswerId = widgets.Dropdown(options=['1','2','3','4'],  description='', disabled=False)

    tb_question = widgets.Text(description='', layout=Layout(padding='0px', height='auto'))
    tb_answer1 = widgets.Text(description='', layout=Layout(padding='0px', height='auto'))
    tb_answer2 = widgets.Text(description='', layout=Layout(padding='0px', height='auto'))
    tb_answer3 = widgets.Text(description='', layout=Layout(padding='0px', height='auto'))
    tb_answer4 = widgets.Text(description='', layout=Layout(padding='0px', height='auto'))

    # Error widgets for each field
    error_widgets = [widgets.HTML('', layout=widgets.Layout(width='20px')) for _ in range(6)]

    btn_insert = widgets.Button(description='Insert', disabled=False, button_style='success', layout=Layout(margin="20px 0px 0px 140px"))
    btn_insert.on_click(insert_new_question_to_db)

    error_label = HTML('')
    success_message = HTML('')  # New widget for the success message

    question_form_layout = VBox(
        children=[
            HBox([HTML('<p style="width:150px; color: #FFFFFF;">Question:</p>'), tb_question, error_widgets[0]]),
            HBox([HTML('<p style="width:150px; color: #FFFFFF;">Answer #1:</p>'), tb_answer1, error_widgets[1]]),
            HBox([HTML('<p style="width:150px; color: #FFFFFF;">Answer #2:</p>'), tb_answer2, error_widgets[2]]),
            HBox([HTML('<p style="width:150px; color: #FFFFFF;">Answer #3:</p>'), tb_answer3, error_widgets[3]]),
            HBox([HTML('<p style="width:150px; color: #FFFFFF;">Answer #4:</p>'), tb_answer4, error_widgets[4]]),
            HBox([HTML('<p style="width:150px; color: #FFFFFF;">Correct answer:</p>'), cb_correctAnswerId, error_widgets[5]]),
            error_label,
            success_message,  # Display success message here
            btn_insert
        ],

        layout=Layout(justify_content='center', align_items='center', margin='30px'),
        style={'description_color': '#FFFFFF'}
    )
    # question_form_layout.layout.width = '60%'
    add_questions_title = widgets.HTML(value="<h1 style='color: #FFD43B; text-align: center;'>Add Questions Page</h1>")
    add_questions_description = widgets.HTML(value="<p style='color:#FFFFFF; text-align: center;'>Please enter the new question and select the correct answer.</p>")

    display(VBox([add_questions_title, add_questions_description, question_form_layout]))

    show_go_back(show_menu_page)

    background_html = """
    <style>
    body {
        background-color: #272822;
        color: #FFFFFF;
        font-family: 'Courier New', monospace;
    }
    .widget-vbox {
        background-color: #3E3D32;
        border-radius: 10px;
    }
    </style>
    """
    display(HTML(background_html))



##########################################################

question_section = widgets.VBox()
buttons = []

def handle_save_click(button, question, question_text_input, answers_inputs, correct_answer_input):
    updated_question = question_text_input.value.strip()
    updated_answers = [answer_input.value.strip() for answer_input in answers_inputs]
    updated_correct_answer = int(correct_answer_input.value.strip())

    # Update the question and answers in the Firebase database
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH, None)
    res = FBconn.get('/Questions', None)

    count = -1
    if res:
        for key in res:
            count = count + 1
            if key:

              if key['Question'] == question.get_question_str():
                  update_url = f'/Questions/{count}'
                  data = {
                      'Question': updated_question,
                      'Option1': updated_answers[0],
                      'Option2': updated_answers[1],
                      'Option3': updated_answers[2],
                      'Option4': updated_answers[3],
                      'Correct': updated_correct_answer
                  }
                  FBconn.patch(update_url, data=data)
                  print("Question updated successfully in the database.")
                  break
        else:
            print("Failed to find the question in the database.")
    updatedData=[]
    # Update the question, answers, and correct answer in the local list
    for module in questionsFromDB.get_all_questions():
        if module.get_question_str() == question.get_question_str():
            module.set_question_str(updated_question)
            module.set_answers(updated_answers)
            module.set_correct(updated_correct_answer)
            updatedData = module
            break
    else:
        print("Failed to find the question in the local list.")

    # After saving the changes, display the question with the updated values
    display_question(updatedData, button)

    # admin_description = widgets.HTML(value="<p style='text-align: center;'>Welcome to the Admin Edit Page. Here, you can choose a question to edit or remove.</p>")

    # # Display a success message for the edit
    # success_message = widgets.HTML(value="<div style='text-align: center; background-color: #d4edda; width: 400px; color: #155724; border: 1px solid #c3e6cb; padding: 10px;'>Question successfully edited!</div>")
    # admin_title = widgets.HTML(value="<h1 style='color: #336699; text-align: center;'>Admin Edit Page</h1>")
    # admin_layout = widgets.VBox(
    #     children=[admin_title, admin_description,success_message, widgets.HBox([buttons_box, question_section])],
    #     layout=widgets.Layout(justify_content='space-around', align_items='center', margin='30px')
    # )
    # clear_output(wait=True)
    # display(admin_layout)
    # show_go_back(show_menu_page)
    # display(HTML(background_html))

    # # Clear the success message after a few seconds
    # time.sleep(2)
    # clear_success_message(success_message, admin_title, question_section, buttons_box)
    # show_go_back(show_menu_page)
    # display(HTML(background_html))
    success_output = widgets.Output()

    # Display the success message
    with success_output:
        display(HTML("<div style='text-align: center; background-color: #c3e6cb; color: #155724; border: 1px solid #b1dfbb; padding: 10px;'>Question successfully modified!</div>"))

    # Clear the output after a few seconds
    def clear_success_message():
        with success_output:
            clear_output()

    display(success_output)
    time.sleep(2)
    clear_success_message()

    # Refresh the admin edit page
    AdminEditPage(None)


def clear_success_message(success_message, admin_title, question_section, buttons_box):
    admin_description = widgets.HTML(value="<p style='text-align: center;'>Welcome to the Admin Edit Page. Here, you can choose a question to edit or remove.</p>")

    admin_layout = widgets.VBox(
        children=[admin_title,admin_description, widgets.HBox([buttons_box, question_section])],
        layout=widgets.Layout(justify_content='space-around', align_items='center', margin='30px')
    )
    clear_output(wait=True)
    display(admin_layout)

def handle_edit_click(button, question):
    # Retrieve the current question and answers
    question_text = question.get_question_str()
    answers = question.get_all_answers()
    correct_answer = question.get_correct_inx()

    # Create input fields for editing
    question_text_input = widgets.Text(
        value=question_text,
        description='Question:',
        layout=widgets.Layout(width='400px', margin='0 0 10px 0', background_color='#000000'),
        style={'description_width': 'initial', 'color': '#FFFFFF'}
    )
    answers_inputs = []
    for i, answer in enumerate(answers):
        answer_input = widgets.Text(
            value=answer,
            description=f'Answer {i+1}:',
            layout=widgets.Layout(width='400px', margin='0 0 10px 0', background_color='#000000'),
            style={'description_width': 'initial', 'color': '#FFFFFF'}
        )
        answers_inputs.append(answer_input)

    # Create a Dropdown widget for the correct answer
    correct_answer_input = widgets.Dropdown(
        options=[str(i) for i in range(1, 5)],
        value=str(correct_answer),
        description='Correct Answer:',
        layout=widgets.Layout(width='400px', margin='0 0 10px 0', background_color='#000000'),
        style={'description_width': 'initial', 'color': '#FFFFFF'}
    )

    # Create a "Back" button to go back to the question and answers
    back_button = widgets.Button(description='Back', button_style='info')
    back_button.on_click(lambda _, btn=button, q=question: display_question(q, btn))

    # Create a "Save" button for saving the changes
    save_button = widgets.Button(description='Save', button_style='success', layout=widgets.Layout(width='120px'))
    save_button.on_click(lambda _, btn=button, q=question, qti=question_text_input, ai=answers_inputs, cai=correct_answer_input:
                         handle_save_click(btn, q, qti, ai, cai))

    # Create a box to hold the input fields, the save button, and the back button
    buttons_box = widgets.HBox([back_button, save_button], layout=widgets.Layout( margin='10px 0'))

    # Create a box to hold the input fields and the buttons
    edit_box = widgets.VBox(
        [question_text_input] + answers_inputs + [correct_answer_input, buttons_box],
        layout=widgets.Layout(padding='20px', width='600px', max_width='600px', word_wrap='break-word', background_color='#000000', border='1px solid #000000',margin='10px')
    )

    # Display the edit box
    question_section.children = [edit_box]
    question_section.layout.visibility = 'visible'




def display_question(question, button):
    question_text = question.get_question_str()
    answers = question.get_all_answers()
    correct_answer = int(question.get_correct_inx())

    question_html = f"<h3 style='color: #FFD43B;'>Question: \"{question_text}\"</h3>"
    answers_html = "<ul>"
    for i, answer in enumerate(answers):
        if i + 1 == correct_answer:
            answers_html += f"<li style='color: #A6E22E;'>{answer} (Correct Answer)</li>"
        else:
            answers_html += f"<li style='color: #FFFFFF;'>{answer}</li>"
    answers_html += "</ul>"

    edit_button = widgets.Button(description='Edit', button_style='success', icon='pencil')
    edit_button.on_click(lambda _, btn=edit_button: handle_edit_click(btn, question))

    remove_button = widgets.Button(description='Remove', button_style='danger', icon='trash')
    remove_button.on_click(lambda _, btn=remove_button: handle_remove_click(btn, question))

    button_box = widgets.HBox([remove_button, edit_button], layout=widgets.Layout(justify_content='center'))

    question_box = widgets.VBox(
        children=[widgets.HTML(value=question_html), widgets.HTML(value=answers_html), button_box],
        layout=widgets.Layout(padding='10px', width='600px', max_width='600px', word_wrap='break-word',border='1px solid #000000')
    )

    question_section.children = [question_box]
    question_section.layout.visibility = 'visible'

    for b in buttons:
        b.button_style = ''  # Reset all button styles
    button.button_style = 'primary'  # Set clicked button style


def handle_button_click(button):
    question_index = buttons.index(button)
    selected_question = questionsFromDB.get_all_questions()[question_index]
    display_question(selected_question, button)

#sharonnew2
def handle_remove_click(button, question):
    _ = questionsFromDB.remove_question_from_DB(question)

    # Remove the button from the UI and the buttons list
    for btn in buttons_box.children:
        if btn.description == question.get_question_str():
            buttons_box.children.remove(btn)
            break

    # # Remove the question section from the UI
    question_section.children = [child for child in question_section.children if not isinstance(child, widgets.VBox)]

    # admin_description = widgets.HTML(value="<p style='text-align: center;'>Welcome to the Admin Edit Page. Here, you can choose a question to edit or remove.</p>")

    # # Display a success message for the remove
    # admin_title = widgets.HTML(value="<h2>Admin Edit Page</h2>")
    # admin_layout = widgets.VBox(
    #     children=[admin_title, admin_description, success_message, buttons_box, question_section],
    #     layout=widgets.Layout(justify_content='space-around', align_items='center', margin='30px')
    # )
    # clear_output(wait=True)
    # display(admin_layout)
    # show_go_back(show_menu_page)
    # display(HTML(background_html))
    # # Clear the success message after a few seconds
    # success_message = widgets.HTML(value="<div style='text-align: center; background-color: #f8d7da; color: #721c24; width: 400px; border: 1px solid #f5c6cb; padding: 10px; margin: auto;'>Question successfully removed!</div>")
    # display(success_message)
    # time.sleep(2)
    # AdminEditPage(None)
    # Create a modal dialog for the success message
    success_output = widgets.Output()

    # Display the success message
    with success_output:
        display(HTML("<div style='text-align: center; background-color: #f8d7da; color: #721c24; border: 1px solid #f5c6cb; padding: 10px;'>Question successfully removed!</div>"))

    # Clear the output after a few seconds
    def clear_success_message():
        with success_output:
            clear_output()

    display(success_output)
    time.sleep(2)
    clear_success_message()

    # Refresh the admin edit page
    AdminEditPage(None)

    # clear_success_message(success_message, admin_title, question_section, buttons_box)


def AdminEditPage(b):
    global buttons_box, buttons
    buttons = []
    output.clear()

    for i, question in enumerate(questionsFromDB.get_all_questions()):
        question_button = widgets.Button(
            description=f'Question {i+1}',
            layout=widgets.Layout(width='100px', height='40px')
        )
        question_button.on_click(lambda _, btn=question_button: handle_button_click(btn))
        buttons.append(question_button)

    # Display only the first 10 buttons
    buttons_box = widgets.VBox(buttons)

    buttons_container = widgets.VBox(
        [buttons_box],
        layout=widgets.Layout(height='400px', overflow_y='auto')
    )

    button_wrapper = widgets.HBox([buttons_container], layout=widgets.Layout(justify_content='center'))

    admin_title = widgets.HTML(value="<h1 style='color: #FFD43B; text-align: center;'>Admin Edit Page</h1>")
    admin_description = widgets.HTML(value="<p style='text-align: center; color: #FFFFFF;'>Welcome to the Admin Edit Page. Here, you can choose a question to edit or remove.</p>")

    admin_layout = widgets.VBox(
        children=[admin_title, admin_description, widgets.HBox([button_wrapper, question_section])],
        layout=widgets.Layout(justify_content='center', align_items='center', padding='30px', width='80%', margin='auto')
    )

    background_html = """
    <style>
    body {
        background-color: #272822;
        color: #FFFFFF;
        font-family: 'Courier New', monospace;
    }
    .widget-vbox {
        background-color: #3E3D32;
        border-radius: 10px;
    }
    .widget-label {
        color: white !important;
    }
    </style>
    """




    display(HTML(background_html))
    display(admin_layout)
    show_go_back(show_menu_page)







##########################################################################################################################
######################################################### LEADERBOARD ####################################################
##########################################################################################################################
def generate_leaderboard():
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
    scores = FBconn.get('/Scores', None)

    if scores:
        sorted_scores = sorted(scores.values(), key=lambda x: x['Points'], reverse=True)

        # Create a list of unique leaderboard entries with 'Username' and 'Points'
        leaderboard = []
        usernames = set()

        for score in sorted_scores:
            username = score['Username']
            points = score['Points']
            if username not in usernames:
                leaderboard.append((username, points))
                usernames.add(username)

        # Find the position of the current user
        current_user_position = None
        current_username = current_user.getName()
        if current_username != "Guest":
            for i, entry in enumerate(leaderboard, start=1):
                username, _ = entry
                if username == current_username:
                    current_user_position = i
                    break

        # Trim the leaderboard to include only the top 10 entries
        leaderboard = leaderboard[:10]

        # Generate the HTML table for the leaderboard with styling
        leaderboard_html = '''
        <table style="width: 50%; margin: 0 auto; text-align: center; border-collapse: collapse;">
            <tr style="background-color: #eee;">
                <th style="padding: 10px;">Rank</th>
                <th style="padding: 10px;">Username</th>
                <th style="padding: 10px;">Points</th>
            </tr>
        '''

        for rank, entry in enumerate(leaderboard, start=1):
            username, points = entry
            row_style = ''
            if current_user_position and rank == current_user_position:
                row_style = 'style="font-weight: bold; background-color: #ffeecc;"'
            leaderboard_html += f'''
            <tr {row_style}>
                <td style="padding: 10px;">{rank}</td>
                <td style="padding: 10px;">{username}</td>
                <td style="padding: 10px;">{points}</td>
            </tr>
            '''

        leaderboard_html += '</table>'

        if current_user_position:
            if current_user_position > 10:
                leaderboard_html += f'<p style="text-align: center; font-weight: bold;">Your position: {current_user_position}</p>'

        return leaderboard_html
    else:
        return 'No scores found'



##########################################################################################################################
########################################################## MENU-PAGE #####################################################
##########################################################################################################################
def show_menu_page(b=None):
    """
    Present the main menu
    """
    clear_output()
    get_questions_from_db()
    # Define image and buttons container
    buttons_container = widgets.VBox()

    # Define image
    image_url = 'https://png.pngtree.com/thumb_back/fh260/background/20210310/pngtree-cartoon-natural-landscape-background-image_581066.jpg'
    image_widget = widgets.Image(value=requests.get(image_url).content, format='jpg', width=50, height=50)

    # Define buttons
    btn_start_game = widgets.Button(description="Start Game", button_style="success", layout=widgets.Layout(width='200px', height='50px'))
    btn_classes = widgets.Button(description="Classes", button_style="info", layout=widgets.Layout(width='200px', height='50px'))
    btn_leaderboard = widgets.Button(description="Leaderboard", button_style="warning", layout=widgets.Layout(width='200px', height='50px'))
    btn_help = widgets.Button(description="Help", button_style="primary", layout=widgets.Layout(width='200px', height='50px'))
    btn_admin = widgets.Button(description="Add Questions", button_style="info", layout=widgets.Layout(width='200px', height='50px'))
    btn_go_back = widgets.Button(description="Back", button_style="primary", layout=widgets.Layout(width='200px', height='50px'))
    btn_admin_edit_questions = widgets.Button(description="Edit Questions", button_style="warning", layout=widgets.Layout(width='200px', height='50px'))
    btn_continue = widgets.Button(description="Continue", button_style="warning", layout=widgets.Layout(width='150px', height='40px'))
    btn_logout = widgets.Button(description="Logout", button_style="danger", layout=widgets.Layout(width='200px', height='50px'))
    centered_container = widgets.HBox(
        [widgets.Label()],
        layout=widgets.Layout(justify_content='center')
    )

    centered_container.children = [btn_continue]
    # Define button callbacks
    def on_continue_clicked(b):
        global list_of_stats
        output.clear()
        list_of_stats = [0]*(questionsFromDB.get_original_len())
        show_questions_page()

    def on_leaderboard_clicked(b):
        output.clear()
        display(HTML("<div style='text-align: center; justify-content: center; align-items: center;'>"
                     "<h2 style='color: red'><img src='https://media.tenor.com/8QEKfyK2RUcAAAAM/rotating-star.gif' "
                     "style='width: 50px; height: 50px'/>Leaderboard! Top 10"
                     "<img src='https://media.tenor.com/8QEKfyK2RUcAAAAM/rotating-star.gif' "
                     "style='width: 50px; height: 50px'/></h2></div>"))
        display(HTML(generate_leaderboard()))
        show_go_back(show_menu_page)

    def on_start_game_clicked(b):
        output.clear()
        show_explanation_screen()

    def on_classes_clicked(b):
        output.clear()
        video_url = "https://www.youtube.com/embed/_uQrJ0TkZlc"

        # Define the HTML code for the Classes page
        html_code = """
        <style>
            body {{
                background-color: #272822;
                color: #FFFFFF;
                font-family: 'Courier New', monospace;
                display: flex;
                align-items: center;
                justify-content: center;
                min-height: 80vh;
                padding: 20px;
            }}
            .classes-container {{
                text-align: center;
                padding: 30px;
                background-color: #3E3D32;
                border-radius: 10px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
            }}
            .classes-title {{
                font-size: 32px;
                margin-bottom: 10px;
                color: #FFD43B;
            }}
            .classes-description {{
                font-size: 18px;
                margin-bottom: 20px;
                color: #FFFFFF;
            }}
            .code-phrase {{
                color: #FF6AC1;
                font-style: italic;
            }}
        </style>
        <div class="classes-container">
            <h2 class="classes-title">Welcome to Python Classes!</h2>
            <p class="classes-description">Unlock your <span class="code-phrase">Python potential</span> with our exceptional <span class="code-phrase">classes</span>.</p>
            <p class="classes-description">Expand your skills and master <span class="code-phrase">Python programming</span>.</p>
            <p class="classes-description">Contact our customer service team to arrange payment and enroll in our <span class="code-phrase">classes</span>.</p>
            <iframe class="classes-video" src="{}" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>
        </div>
        """.format(video_url)

        # Display the Classes page
        display(HTML(html_code))
        show_go_back(show_menu_page)

    def show_explanation_screen():
        output.clear()
        image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Python.svg/1200px-Python.svg.png'

        html_code = """
        <style>
            body {{
                background-color: #2B2B2B;
                color: #ECECEC;
                font-family: 'Courier New', monospace;
                display: flex;
                align-items: center;
                justify-content: center;
                min-height: 80vh;
                padding: 20px;
            }}
            .explanation-container {{
                text-align: center;
            }}
            .explanation-image {{
                width: 200px;
                height: 200px;
                margin-bottom: 20px;
            }}
            .explanation-title {{
                font-size: 32px;
                margin-bottom: 10px;
                color: #FFD43B;
            }}
            .explanation-text {{
                font-size: 18px;
                margin-bottom: 20px;
                color: #ECECEC;
            }}
        </style>
        <div class="explanation-container">
            <img src="{}" class="explanation-image">
            <h2 class="explanation-title">Welcome to the Game Explanation!</h2>
            <p class="explanation-text">In this game, your Python knowledge will be put to the test.</p>
            <p class="explanation-text">Answer challenging Python trivia questions to earn points.</p>
            <p class="explanation-text">On the first try, correct answer gives you 3 points.</p>
            <p class="explanation-text">On the second try, correct answer gives you 2 points.</p>
            <p class="explanation-text">On the third try, correct answer gives you 1 point.</p>
            <p class="explanation-text">No points are awarded on the fourth try.</p>
            <p class="explanation-text">Get ready to have fun and sharpen your Python skills!</p>


        </div>
        """.format(image_url)
        display(HTML(html_code))
        display(centered_container)
        show_go_back(show_menu_page)



    def on_help_clicked(b):
        output.clear()
        image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Python.svg/1200px-Python.svg.png'

        # Define the HTML code for the Help page
        html_code = """
        <style>
            body {{
                background-color: #2B2B2B;
                color: #ECECEC;
                font-family: 'Courier New', monospace;
                display: flex;
                align-items: center;
                justify-content: center;
                min-height: 80vh;
                padding: 20px;
            }}
            .help-container {{
                text-align: center;
            }}
            .help-image {{
                width: 200px;
                height: 200px;
                margin-bottom: 20px;
            }}
            .help-title {{
                font-size: 32px;
                margin-bottom: 10px;
                color: #FFD43B;
            }}
            .help-text {{
                font-size: 18px;
                margin-bottom: 20px;
                color: #ECECEC;
            }}
            .help-fun {{
                font-size: 24px;
                color: #FFD700;
            }}
        </style>
        <div class="help-container">
            <img src="{}" class="help-image">
            <h2 class="help-title">Welcome to the Help Page!</h2>
            <p class="help-text">In this game, your Python knowledge will be put to the test.</p>
            <p class="help-text">Answer challenging Python trivia questions to earn points.</p>
            <p class="help-text">On the first try, correct answer gives you 3 points. </p>
            <p class="help-text">On the second try, correct answer gives you 2 points. </p>
            <p class="help-text">On the third try, correct answer gives you 1 point. </p>
            <p class="help-text">No points are awarded on the fourth try.</p>
            <p class="help-fun">Get ready to have fun and sharpen your Python skills!</p>
        </div>
        """.format(image_url)

        # Display the Help page
        display(HTML(html_code))
        show_go_back(show_menu_page)
    def on_logout_cliecked(b=None):
        global current_user; current_user = None
        show_login_page()

    # Assign callbacks to buttons
    btn_start_game.on_click(on_start_game_clicked)
    btn_classes.on_click(on_classes_clicked)
    btn_help.on_click(on_help_clicked)
    btn_admin.on_click(show_add_questions_page)
    btn_leaderboard.on_click(on_leaderboard_clicked)
    btn_admin_edit_questions.on_click(AdminEditPage)
    btn_continue.on_click(on_continue_clicked)
    btn_logout.on_click(on_logout_cliecked)


    # Add buttons to the buttons container
    if current_user.isAdmin():
        buttons_container.children = [btn_start_game, btn_leaderboard, btn_classes, btn_help, btn_admin_edit_questions, btn_admin,btn_logout]
    else:
        buttons_container.children = [btn_start_game, btn_leaderboard, btn_classes, btn_help, btn_logout]
    buttons_container.layout.align_items = 'center'

    # Generate the HTML code
    html_code = """
    <style>
        body {{
            background-image: url("{}");
            background-size: cover;
            background-position: center;
            display: flex;
            align-items: center;
            justify-content: center;
            min-height: 90vh; /* Updated value: Increase the min-height for a larger page */
            padding: 20px;
        }}
        .widget-button {{
            border-radius: 20px !important;
        }}
        h1, h3 {{
            color: #000000;
        }}
    </style>
    <div>
        <h1 style="text-align: center;">Welcome to Python Trivia Game!</h1>
        <h3 style="text-align: center;">Challenge your Python knowledge and become a Python master!</h3>
    </div>
    """.format(image_url)



    # Display Menu
    display(HTML(html_code))
    display(buttons_container)


##########################################################################################################################
########################################################## LOGIN #########################################################
##########################################################################################################################


def is_user_Exist(username):
  FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
  users = FBconn.get('/Users/', None)
  for user_id, user_data in users.items():
      if user_data['Username'] == username:
          return True
  return False


def validate_login(username, password):
      """
      Validate the login info against DB
      """
      def get_user():
        global current_user
        FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
        users = FBconn.get('/Users/', None)

        for user_id, user_data in users.items():
            if user_data['Username'] == username and user_data['Password'] == password:
                current_user = User(username, password, user_data['Role'], user_data['TotalPoints'])
                return True
        return False

      if not username or not password:
          show_h4_center_text("Please insert username & password")
      else:
          res = get_user()
          if res:  # user and password corrects
            show_menu_page()
          else:
            show_h4_center_text("The username / password are incorrect")
            return



def register_page(b=None):
    """
    Create and show the registration page elements
    """
    output.clear()
    global show_go_back
    def handle_btn_clicked(b):
      sign_up(tb_username.value, tb_password.value, confirm_password.value)

    tb_username = widgets.Text(placeholder='Username')
    tb_password = widgets.Password(placeholder='Password')
    confirm_password = widgets.Password(placeholder='Confirm Password')
    side_img = widgets.HTML('<img style="width: 150px; height: 150px" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAOEAAADhCAMAAAAJbSJIAAABj1BMVEX/////x4zyURlN1t4KWF4nvsr1YyIFO0DuWh/5+Pj/4bL3URYATlQAWGCwUzdCv8cARk0znqUANDkegIdC1N3luoXDUSjxWBjz/f3Jc1DoXiY2uL237fEjkZndYCji+Plu3OPUgVr4YBrS8fP/xopUyNHtSQDi6uu/ztDMkXhvztaygWel3uL/zZH0VgCE4ef+7ujxPQDyhWYAMTz+5tr82s72ekn6tZ7xcDnzZDP1Zyj1f1r70MH839T3mHf5qpP0dVL6nGb5jlX/1KD9s3kWZ235vKv2lHf0b0X8qXD2hl32lH73gVL1lW/+v4blxpZgv73Sr4D+yqGjuLo/c3eJo6QAIilkiItJZ2q1kYKwSindbkD6yLnefUz3fURcrrB1l4mExLiZi2wmR0f0on2di2uVwrFRYlaqwKiAfGXDo3iXp5vRwJdgaFq/waA6WVF9z8i/yq+mn4CDtKZyhHQASlz6o3OEeFzLrIFYcWUAJTj+yZ3+2bT/6tE1d3PAxcB7XFCRXEB/jIFsYVO1bkgkbTd+AAASA0lEQVR4nN2diV8TZxrHZ0YSEpM0iKZpbd3adJy6x6QJBAgBBQxQiRDFo7pbl10qeFCPag+3hda2u3/4vnNl5r1m3msS6e/Tokxikm+e532O931nRtM4lR+xeD8vJ92t28WJEWvtzqfpAd4aK46NXKVi8U5agHe+GB81nqfi3XQAb30xPj4h/+mKCvwgHSvmJ8eBJD9ece3UB6fW5Bkn0hiLt1xCST89fcLRaWnCsdspEN7zCOUQT3iSJ5xQD5h/4AHKDMXiOZ/wnLSfFi8qJ/x0PJD4pyue8glPvd2EEoinfUL5gZguocRQLLuAZWnAtAnFEdccxPLa208oEW3WTivgS59QQW3zthPK1jbHgHDUNfgQCCfcInpYGgXh+MTpU8PTuVEQTp4+c2JoKqNWHA5h5o9PODzEkREODXF0hBf+8ISZIUWbERIOyU9HSTgcRJzw7+oJJymE5ZEQlv6xoBiw8SXNhkMZijjhO835hkrAVudPVMJhIBIICwXjkjrAmVothnAIQ5FEaBSsGVWAi7VsNo4wfUQiIUBcVARYzyYQpp74yYSGYd1XBhhPmPpQpBEaTQWIHmACYdp+SiVUgOgDJhGmjEgnNGTH4kwty0aoBJHq7DGEhlxEbQWAyYThpzsv/HYf0hDjCAuWRF5sdNgJB4hnPhsuoVEwxKub9gCQgTAzKkKjMC/6hr0QkIUwMypCwxIsw/fqWT7CzKgIDeuayNutRwHZCC+MitBorvO/2yUIkI3QjTYjITSa3Dkj36kJEGZGRlio8gbUHgzISpgZFaFhLfO91xzso+yE5VERGs0lnreaQQGZCcFQHBEh11DMt2vChJlR2RAkfvbNp5cxE3IQZs6/J6rPpAgN63NWwEUckIcwhU6KjdBosnZSHRyQi1B9x89IaFiMPooNQk5C9YjMhFssgJcIPspJqNxPWQnZ4uksyYSchKoRmQkLV5MB14km5CVUvJrBTGhYc0mADSIfN6HiochOaFhJ9eke0Uf5CTNnBKSEMCEptsg+ykf4N1cf8uu8AkKj2YolRFsKEcK/aKKn78hVbYERY6c0yJmCl1D4FCUlhPEZg5wpjhlhXMYgFaQShNyIagjjJvo3qSYUI+RFVERYWKa9Ad73yhJyIioipBuRGkiPHSElnFJzoQQhH6IqQqNJXqwhdk2yhFyIygitKdLLN+JMKE7Ig6iM0GiSqlNsAlERIQeiOkKLNLVImrsQI9RGT0iaz4jJ9pyEfuUdariVt++m+A6GuFTBSYhpmN1TYEMsYTTiAWv/nJAglJ7U4Cc0CmisiY8z2WxFxobSHb8AITadEVOSOiZckSOURRSxIdJhxCfDbMeWJJT0UwFCw4J7/XgnrW1IE8ohChHCbhrvpG1blyaU8lMRQthN4520tqrLEzKlDJWEcOVGmQb2Aa/YXITlT8j6KFGSq2txbhqf7vs6FyFal7KXbwqrNgNp9RNNqIQwEVEtodEMXzlu+sIzoRrCJETVhOFkxlKMk9ZW7ONKGGmh6NOkINmbujrCBETFhOGexXyyCVURxiMqJgzXoeKGYUfXlRLGIionDAZiTMkG6rXjTBgMxJhs2NZV2zAOUTVhIWiD28kmVEgYg6ia0PB3ScW09wMTqiSkIyon9Dso+hxUaEIuwqRrKw6P0M/59EDTtkVsmLmQpPMUUfgkbOiFGupsfsSECrqnqHibRWHCz+NDaScEVEzI2/GLEnrtRZ4WSoNyJg1CTkRRQi+YUkNpR9fTI+TzU2FCt26jLRtCJlROyIcoTOhurqEmi36aNuTzU3Eb3qcnC6+1T5GQB1Gc0JmrobW/q3rKhByI4oROQqTs1duxUydkH4rihM4pX1tEwhpswlQI2Y0oTOh2F+Tp7o49BEJmRHHCq7TeCU4VqRGyIooTOlM1xPX7mqkPhZARUZjQqAJCkpPWNu0hEbJFG3HCukae70bjTHqEAJFhlUac0KIQdlAnTSL81/vC+piiXSWETTIhUs8wEP61XhVWjaj6n88oIcxreRIh5qSJhNWcsEiRDnyGVAlxJ8UIp8f/vbtbVkJIRkyVkOCkKOH0VzZ4jlnZpRI6/uf/UfV/d354f/WPV/2HHa9Mj5AwDglOihBOP7Htk0C2uUshrF6dmprqWLkpR7mclZ2dWuhUF6bmc/U2+NtZ9/iC93B1tjfVa9eHSNjBARHCB7oLCBArFELLmT7I73mrP1lryem1F60Z7bLlXBtgxttF0Nhx/3BnNfNIj6OKkBhLd3AnhQmn9/WuR9g1b1AJV/a0fE5rbPV6ddClzfXmrgHCrbbW6O3t5bbAg71eG/zY2qq3tK0trdFJg9Ai1TRYTYrbsOKb8ORJ/SGVsL2ltQBhr7dZbWiz1s7Ojkeo3d+0clUAZzk/ZnuzgHB5Fp0vUkRYJ9WltY0EL518ZHcHhI/jvLTtemkL+GSz43yZgNCaA48cWQGh87A39z6bCmGV1FvU+gmEoZPGEl5uaUvzWmNqajkHfrGWG1odEFbr9R5404ENF6aWgQ2vzaA7XtQQur0F3h8SnBQmrOgnGQg7U46X5tfX57JgHM4A27mE+cX7mlYdEM6trwMbLu+gu5YUEV4l9fjtJMIJ2z6ZOA6BnYBjzrrBsm0Be2r5GUC41wZ/u7TgEGq+l2pWQ1uw0K9aEeECaZ6GFEqjhEEydOUnRDzjW1bV+d9yBNKj++fZK6u6bW5krap7JDgcPDcFG7rzNNhcW5INpw/CYdilEhLkxmjn/x7l2SkQLhHmS/HuFyXUo4SUfEjQqm26tu/q9g7lKeoJ5whz3rXZBMIXoCAN0oVtlhkJqyu2GZQJep/2dOWE64R1iyQbgqIb/A66D1B7mzorYRX0K4PR29U3Ev1UDaG7boGuPZE6C4hwG3rAI7yTO5ugnYEJnQisz1Ke1gmU/VqJDRuE9cMkwnH4uN8+fZCkXTNC2DWf0p534cKFEye8/xQQepsxkDVgYlkaEk4+gx6+zjjhlLkRJTyp+xGKoBMEya0Bo+v4CYTT+2KEZT1K2DWf055Iml+UXMdH0kVthQAYIaxED5tPWQkz182wTgCVgkl9okpCby8GsnGP2FqEhA90QcKXphkhtM2HHIiS+2nQxiyWEBmGdGfDBIUa4KYx/q3Ohv5ZJXAwJc3ShISgZIMIXzITZmw94qZ6TKzBh6LkvjYkmBLbw9BLKzAh3dcwPTf10ILx5kcRZfcmwr0ntuwEEb6whQkH+aKre29Rpj9XDeFgfykSakiAAeH0a4Rwl/4pMV0HlV63a5smw7ejhjDYIwzv8ybNJYaE8DCMG0y4HoIy1ox4SFwuLSshHJwXBE0CtXWSAkLkMBdh2eT4xwpW1yLXF15iJkSGIR9h5imMGJ9qzkgTRs63iA5EcuHtE3qdU1Qx0QLXLhrEYv+1NGH0IoPMhBXksM1FmEE9ID4SSxOGgNH5NnLh7RNOYoe5AJ2UCCm+br8gRwiduxbJiOTC2yWcHD9ADzO3Fp5u8MQagPjNN9619gPCEg8hdP5hZDaWXJY6hNOPKph5OQmdlAgRRmNN+cZuNLveeO7kFvN5QFgqlfwfjITw1T/Cwo1clgLC6SeEw7yEj1EjDvAeX3d4BiPzxnU/c5rXPcLS2Ffbuq0f3CsxEiKnq4c9IrksBYQHhPHJ0Tz5IOgLeETl54NC5zH6TZgvzwDC4hN3phX8+LHERoicyx1xUwrhNikAcTRPvp6SnAAYLHzJG6ipzfMnyhOvwrd3RiP/+fjhogFhlwJd/jfOITQlukB29GAFDUjAiOUnke/3oChyTYUwmpJLGmWEGfQVnrv1avTIQzSpmGcy0AzmWknkuhiDRp+4LkMl5GiefL1EjAhqBsRzKxjhwwxUabwqilzbZJD0+Qh5midPWEp8iBKau6grdzNQIq6ApMF/fZpg+YJStKkjRFMiiDVopfMUc+Vv70U/lf1dMiHpLkm1uKKNRsjVWnjCUmIZjz6oK1//HhqI20WR60T5KbH2AwegrgsQYl3i8wx2BH2O/uN30K+3ha715aVESklD0zkB/Qd7kVfYkW3kwMH30K+vhK7X5s3ucxJO8mv6CTIO7NePsCPP0LEyBnemSdfcI19M2FtIJJc0NE2P8+sB+iKVn9C2s4LOltj7Y3CsSbhuIuVawk753eEj/EmAEJ3N0vUXr1EjPttHjViEnHs7/tqXxMsmat5kBh9hRcSGk8+QV7EPMLNuo0fs1z9CRhS7fqlTnHIVbfq2CCE+F6LjZn2AxpoKNNturwhdg9bJ+sStJlQdiBGiLmg/QSOLvY/Fmm3oe+kLmdCZOOUq2gQJxx9gnx6LNfo4dgT+CmapRoy9e8ClOlfRZu+LEU6jLqi/QKcp7a/Q6INo1aIRxt8BYnMohJNYSjyYxk7SeZHw5vM0H42/zVzrCgcgCHBihMh+DkdYrLH3MUPDj69QjJhwXX3tJk+Lbz9D79bNKJznCVrXACvGvzkl1iTeMKjBkw+F0qGjSYyHEGsSZG+SY03i/deO2I1oPxI04Ti6lAxe6wG2JJIkYqyxkm8TmGd/C8Fc4QhLifo+VtckySbEGpb7zGgtZiO+EAZ0NzgiSkiABEJCXcN277VDNkTRVOEbEYuUB/GxkyATI2S8L1meLdhgZ7PxEaIpUUB4rKkzAYIeg8WI9sfvykkaEI817PcHZEmKq1ZBShZ5jYtPcKxhv3ceS1IkBTI+8XUx5A8B1TU89z9Mjqf2D9TCl1UW32wCUVBdw3d/7qS8b1ZlAY0C18wsWfZOGGuaifcjg5QQT+0rjCdWxSknT6ivDj4H771kExBX5fmAEfkmLokahINCjvsG660YxKhzSBAST3vgJAzqGv57OmvaIn0obkiHGVc5BbFG9z6KyH25QVa03c0QhFeVzhSerA1VsUbs3uogLeYbP3cPTRTTXqlllRAW+KYuyXLqmoSJC7q8azgCzCMI08xms/LJwkVU4abz0T2I/EYM//rzUdfDtK/URBAL7qmGcIRSkhJBXcMdRomIAaa+Wq/zIxbmVzb6/dWNKzmIEb1Wk4j6Va5aBkO8iNV6jcW5XrvDibhheyy2vTEfYbQUpMS+FKCmvZn45e6ti5++hxyeA2bksOBm1w73zETqIQUpsS+QCFHEYnFi7Pbdby9+FMF0V1PZCWe902S8zwSVfNKArLfijkV0dwMWi2Nrt+9c/MQ76G2hYkccnGXhIrYHiAXJLlEFIEAsDrZ1FoF+AZiNTo0L0frhZNSK/ZBQrktUAwgQ4f25wGuLY7/+9iVPQO0E5wJ5RoxMsMikRPkxGOhiaMXBJt3SM56c0TyE/DScYClwLQUhgAkrFLKI77h+yohYuOKfduh/uLCynRcGNBUCgiR4D9tL/q63D47RiAXvlCffYJFwKpoSzb5EJUNS/i6KeDbLg2itRAkjEyyCKdE8FLlpdLx+hxBLY4Od4WyIOYgwEmtyPOt5A8CbyvmAbkURS/8NT9FgM+JqlDAyDyIycaoqS6CKZo3S/yLnSjEhzkOEoRELxKtUxAMqjTFR5W8PzFh6P0KYYyFs9p1zK8NPGcYaTj7lMQbS10HaKEHnLLLkjEK7C+1qGoRTzi4xnSEY6s1aEQk0zIhoXgj9l4PQTM9DA+XvOJV46VfkeiEsfgrXoJFFB/Zoat5M00MDvQFtRuk39MpELEZEatCgxWBuMEx1hWi88r8XoVDKiohGzSDtF1YYAVPI8jS9WauigExWhI1o+zPLbKFmaAb0tZ7Fr7CcbESkkfCHYqGXTGj2j4ZnQE+NvTrKmBxQc8jntnsOopVsw/7hMCIMqlYPZUxExFjsjZxlXU3qgk099RRB0aXZOmdaRI0IGPt6ggVN2flCKS1u1rmKG/4y29SHG2BwzcC+mmREnvoF4JmHl4YdYAhqXa7XmRF5WnrAN6rxhyo/16kzNsQF5o3kID+MIn5StbiVDSATjMg0fQjMN/MWuCesxvomy6oUw/Sh2dffLvOFasy5kAkBNd6IZh9451tnvoga61v1uhW3VSOmCjVdvFEjMGhm6Sq+3huKPLlmArybb9/Yo6oxs7QwbzWJmOgWDIdNP7zZOg7GQ9Samft82eW0nK2aA8T5AZnjloc3j1qNY0g3UL7Ruj+3dG3h6nyubjU9rfRX+yBeHh0tArbj45gsYrkpsGr9H4+dw6cdIzd3AAAAAElFTkSuQmCC"/>')



    btn_submit = create_widget(WidgetEnum.BUTTON, 'Submit', button_style='success', onclick=handle_btn_clicked, layout=widgets.Layout(margin='20px 0'))
    registration_form = widgets.VBox([tb_username, tb_password, confirm_password, btn_submit], layout=widgets.Layout(align_items='center', justify_content='center'))

    # display(registration_form, justify_content='center')


    side_form = widgets.VBox([side_img], layout=widgets.Layout(margin='30px 0 0 0'))

    border_style = '200px solid #999999'

    split_screen = widgets.VBox([widgets.HBox([registration_form, widgets.HTML('<hr style="border-top: {0}; width: 2px; margin: 0 50px; ">'.format(border_style)), side_form], layout=widgets.Layout(justify_content='center'))])
    split_screen.children[0].layout.border_top = border_style

    html_code = """
    <style>
        body {{
            background-color: #272822;
            color: #FFFFFF;
            font-family: 'Courier New', monospace;
            display: flex;
            align-items: center;
            justify-content: center;
            min-height: 80vh;
            padding: 20px;
        }}
        .main-container {{
            text-align: center;
            padding: 30px;
            background-color: #3E3D32;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
        }}
        .main-title {{
            font-size: 32px;
            margin-bottom: 10px;
            color: #FFD43B;
        }}
        .main-description {{
            font-size: 18px;
            margin-bottom: 20px;
            color: #FFFFFF;
        }}

    </style>
    <div class="main-container">
        <h2 class="main-title">Create new account</h2>
        <p class="main-description">Please insert Username and Password</p>
    </div>
    """.format()
    display(HTML(html_code))
    display(split_screen)
    show_go_back(show_login_page)




    def sign_up(username, password, confirm_password):
      """
      Insert new user to DB
      """
      global current_user

      try:
        if not username or not password or not confirm_password:
          show_h4_center_text("Please fill username and password")
          return
        if password != confirm_password:
          show_h4_center_text("Passwords are not matches")
          return
        if is_user_Exist(username):
          show_h4_center_text("Username already in-use. Please select another")
          return

        data_to_upload = {
          'Username' : f'{username}',
          'Password' : f'{password}',
          'Role' : 'User',
          'TotalPoints': 0
        }
        FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
        result = FBconn.post("/Users", data=data_to_upload)

        display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
        f"<h4>You registered successfully!<br/>Next time you can login with your details</h4>"))
        current_user = User(username,password, "User", 0)
        time.sleep(3)
        show_menu_page() # Auto navigate to

      except Exception as e:
        print(f"Can't insert into DB\n{e}")
        return False


def show_login_page(b = None):
    output.clear()
    # Create login form fields
    login_username = widgets.Text(placeholder='Username')
    login_password = widgets.Password(placeholder='Password')
    login_button = widgets.Button(description='Login', button_style='success', layout=widgets.Layout(margin='20px 0'))
    login_button.on_click(lambda _: validate_login(login_username.value, login_password.value))

    def play_as_guest_init(b):
      global current_user
      current_user = User("Guest", "0", "Guest", -1)
      show_menu_page()

    register_button = widgets.Button(description='Register', button_style='info')
    register_button.on_click(register_page)
    guest_button = widgets.Button(description='Play as guest', button_style='warning')
    guest_button.on_click(play_as_guest_init)

    login_title = widgets.HTML('<h2 style="color:white">Sign-in</h2>')
    or_title = widgets.HTML('<h5 style="text-align: center; color: white; font-family: Arial, sans-serif; font-size: 12; margin-top: 20px;padding:0 50px;">OR</h5>')

    login_form = widgets.VBox([login_title, login_username, login_password, login_button], layout=widgets.Layout(width='500px'))
    side_form = widgets.VBox([register_button, or_title, guest_button], layout=widgets.Layout(margin='30px 0 0 0'))

    border_style = '200px solid #999999'

    split_screen = widgets.VBox([widgets.HBox([login_form, widgets.HTML('<hr style="border-top: {0}; width: 2px; margin: 0 50px; ">'.format(border_style)), side_form], layout=widgets.Layout(justify_content='center'))])
    # Apply border style
    split_screen.children[0].layout.border_top = border_style

    html_code = """
    <style>
        body {{
            background-color: #272822;
            color: #FFFFFF;
            font-family: 'Courier New', monospace;
            display: flex;
            align-items: center;
            justify-content: center;
            min-height: 80vh;
            padding: 20px;
        }}
        .main-container {{
            text-align: center;
            padding: 30px;
            background-color: #3E3D32;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
        }}
        .main-title {{
            font-size: 32px;
            margin-bottom: 10px;
            color: #FFD43B;
        }}
        .main-description {{
            font-size: 18px;
            margin-bottom: 20px;
            color: #FFFFFF;
        }}le

    </style>
    <div class="main-container">
        <h2 class="main-title">Welcome to Python Game!</h2>
        <p class="main-description">Sign-in to keep your statistics and earn rewards, or practice as Guest</p>
    </div>
    """.format()
    display(HTML(html_code))
    # Display the split screen
    display(split_screen)



##########################################################################################################################
########################################################## MAIN ##########################################################
##########################################################################################################################
get_questions_from_db()
show_login_page()



HTML(value='\n    <style>\n        body {\n            background-image: url("https://png.pngtree.com/thumb_ba…

In [ ]:
!pip install firebase

%%capture
!pip install pandas_read_xml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


UsageError: Line magic function `%%capture` not found.


In [ ]:
# import pandas as pd
# import pandas_read_xml as pdx
# from firebase import firebase
# import xml.etree.ElementTree as ET

# FIREBASE_PATH = 'https://ex6game-default-rtdb.europe-west1.firebasedatabase.app/'

# test_xml = """<?xml version="1.0" encoding="UTF-8"?>
# <Questions>
#   <Question>
#     <text>What is the output of the following code?
#     x = [1, 2, 3]
#     print(x[1])</text>
#     <Option1>2</Option1>
#     <Option2>3</Option2>
#     <Option3>[2]</Option3>
#     <Option4>IndexError</Option4>
#     <Correct>1</Correct>
#   </Question>
#   <Question>
#     <text>Which of the following data types is mutable in Python?</text>
#     <Option1>int</Option1>
#     <Option2>float</Option2>
#     <Option3>str</Option3>
#     <Option4>list</Option4>
#     <Correct>4</Correct>
#   </Question>
#   <Question>
#     <text>What is the correct syntax to print "Hello, World!" in Python?</text>
#     <Option1>print("Hello, World!")</Option1>
#     <Option2>print "Hello, World!"</Option2>
#     <Option3>echo "Hello, World!"</Option3>
#     <Option4>printf("Hello, World!");</Option4>
#     <Correct>1</Correct>
#   </Question>
#   <Question>
#     <text>What is the result of the following expression?
#     5 + 3 * 2 - 1</text>
#     <Option1>12</Option1>
#     <Option2>14</Option2>
#     <Option3>16</Option3>
#     <Option4>18</Option4>
#     <Correct>2</Correct>
#   </Question>
#   <Question>
#     <text>Which of the following is not a Python keyword?</text>
#     <Option1>if</Option1>
#     <Option2>for</Option2>
#     <Option3>while</Option3>
#     <Option4>var</Option4>
#     <Correct>4</Correct>
#   </Question>
#   <Question>
#     <text>What is the value of the expression:
#     bool(0) and bool(1)</text>
#     <Option1>True</Option1>
#     <Option2>False</Option2>
#     <Option3>1</Option3>
#     <Option4>0</Option4>
#     <Correct>2</Correct>
#   </Question>
#   <Question>
#     <text>What is the output of the following code?
#     x = [1, 2, 3]
#     x.append(4)
#     print(x)</text>
#     <Option1>[1, 2, 3, 4]</Option1>
#     <Option2>[1, 2, 3]</Option2>
#     <Option3>[4]</Option3>
#     <Option4>4</Option4>
#     <Correct>1</Correct>
#   </Question>
#   <Question>
#     <text>What is the result of the following expression?
#     2 ** 3</text>
#     <Option1>2</Option1>
#     <Option2>3</Option2>
#     <Option3>6</Option3>
#     <Option4>8</Option4>
#     <Correct>4</Correct>
#   </Question>
#   <Question>
#     <text>Which built-in function in Python can be used to sort a list?</text>
#     <Option1>sort()</Option1>
#     <Option2>sorted()</Option2>
#     <Option3>order()</Option3>
#     <Option4>arrange()</Option4>
#     <Correct>2</Correct>
#   </Question>
#   <Question>
#     <text>What is the correct way to declare a variable in Python?</text>
#     <Option1>var x = 5</Option1>
#     <Option2>x = 5</Option2>
#     <Option3>int x = 5</Option3>
#     <Option4>declare x = 5</Option4>
#     <Correct>2</Correct>
#   </Question>
# </Questions>"""

# firebase = firebase.FirebaseApplication(FIREBASE_PATH, None)

# # Parse the XML using ElementTree
# root = ET.fromstring(test_xml)

# # Iterate over each question element and add the questions to Firebase Realtime Database
# question_id = 1
# for question_elem in root.findall('Question'):
#     question_text = question_elem.find('text').text.strip()
#     option1 = question_elem.find('Option1').text.strip()
#     option2 = question_elem.find('Option2').text.strip()
#     option3 = question_elem.find('Option3').text.strip()
#     option4 = question_elem.find('Option4').text.strip()
#     correct_answer = int(question_elem.find('Correct').text.strip())

#     data = {
#         'Question': question_text,
#         'Option1': option1,
#         'Option2': option2,
#         'Option3': option3,
#         'Option4': option4,
#         'Correct': correct_answer
#     }

#     # Push the data to the 'Questions' node in Firebase Realtime Database with the numerical ID
#     response = firebase.put('/Questions', question_id, data)
#     question_id += 1
